This notebook runs the experiments in "Stopping Criteria for Technology-Assisted Reviewsbased on Counting Processes" 

In [1]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import math
from scipy.optimize import curve_fit
import random
import glob
from scipy.integrate import simps
from scipy.stats import norm




# IMPORT EXPERIEMENTAL FUNCTIONS
from utils.read_data_fns import *
from utils.target_method_fns import *  
from utils.knee_method_fns import *  
from utils.inhomogeneous_pp_fns import *   
from utils.eval_fns import *

In [2]:
# READ TOPIC RELEVANCE DATA
with open('data/relevance/qrel_abs_test.txt', 'r') as infile:
    qrels_data = infile.readlines()    
query_rel_dic = make_rel_dic(qrels_data) # make dictionary of list of docids relevant to each queryid
all_runs = glob.glob('data/runs2017_table3/Waterloo/B*')    


# SET POISSON PROCESS PARAMETERS
sample_props = [0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,
                0.7,0.75,0.8,0.85,0.9,0.95,1]  # proportion of docs to sample
min_rel_in_sample = 20 # min number rel docs must be initial sample to proceed with algorithm 
n_windows = 10  # number of windows to male from sample

# SET KNEE METHOD PARAMETERS
knee_rho = 6 # knee method rho 

# SET EXPERIMENTAL PARAMETERS (des_probs are specified below)
des_recalls = [0.95, 0.9, 0.8, 0.7] # desired recalls to experiment over

In [3]:
# fn to run stopping methods (oracle, knee method, target method, poisson process and cox process over all topics)
def run_sp_algorithms(des_recall, des_prob):
    
    # PREPARE SCORING DICTIONARIES
    run_score_dic = {}
    oracle_dic = {}
    
    # LOOP OVER RUNS
    for run in sorted(all_runs):


        # MAKE DATA DICTIONARIES
        with open(run, 'r') as infile:
            run_data = infile.readlines()
        doc_rank_dic = make_rank_dic(run_data)  # make dictionary of ranked docids for each queryid
        rank_rel_dic = make_rank_rel_dic(query_rel_dic,doc_rank_dic) # make dic of list relevances of ranked docs for each queryid

        # PREPARE SCORING DICTIONARIES
        run_name = run[5:]
        score_dic = {}
        oracle_dic[run_name] = []

        # LOOP OVER TOPICS
        topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs
        for query_id in topics_list:
            #print(query_id)
            score_dic[query_id] = []      

            # EXTRACT COUNTS AND REL LISTS
            n_docs = len(doc_rank_dic[query_id])  # total n. docs in topic
            rel_list = rank_rel_dic[query_id]  # list binary rel of ranked docs 


            # ORACLE
            rel_doc_idxs = np.where(np.array(rel_list) == 1)[0]
            orcale_n_rel = math.ceil(len(rel_doc_idxs)*des_recall)
            oracle_idx = rel_doc_idxs[orcale_n_rel-1]
            oracle_eff = oracle_idx+1
            oracle_dic[run_name].append(oracle_eff)
            


            # TARGET METHOD
            random.seed(1)
            target_size = get_target_size(des_recall, des_prob)
            target_list, examined_list = make_target_set(rel_list, n_docs, target_size)  # get target sample and list all docs examined
            tar_stop_n = get_stopping_target(target_list, n_docs, target_size)  # stopping point
            all_examined_idxs = get_all_target_examined_idxs(examined_list, tar_stop_n)  # list of every doc examined during method
            tar_recall = calc_recall(rel_list, tar_stop_n)
            tar_effort = len(all_examined_idxs) # total effort (inc. sampling)
            tar_accept = calc_accept(tar_recall, des_recall)
            score_dic[query_id].append((tar_recall, tar_effort, tar_accept))
            
            
            # KNEE METHOD
            batches = get_batches(n_docs)
            knee, knee_stop = get_knee_stopping_point_var_adjust(rel_list, batches, knee_rho, 150)[0:2]
            knee_recall = calc_recall(rel_list, knee_stop)
            knee_effort = knee_stop
            knee_accept = calc_accept(knee_recall, des_recall)
            score_dic[query_id].append((knee_recall, knee_effort, knee_accept))


            
            # INHOMOGENEOUS POISSON PROCESS EXPONENTIAL (IP-P)
            # check topic meets initial relevance requirement
            n_samp_docs = int(round(n_docs*sample_props[0]))
            sample_rel_list = rel_list[0:n_samp_docs]  # chunck of rel list examined in sample

            # if meet size requirement run algorithm; else return n_docs as stopping point
            if (np.sum(sample_rel_list) >= min_rel_in_sample):

                windows_end_point = 0
                pred_stop_n = n_docs
                i = 0

                while (i < len(sample_props)) and (pred_stop_n > n_samp_docs):
                    sample_prop = sample_props[i]

                    n_samp_docs = int(round(n_docs*sample_props[i]))
                    sample_rel_list = rel_list[0:n_samp_docs]  # chunck of rel list examined in sample

                    # get points
                    windows = make_windows(n_windows, sample_prop, n_docs)
                    window_size = windows[0][1]

                    x,y = get_points(windows, window_size, sample_rel_list)  # calculate points that will be used to fit curve

                    good_curve_fit = 0
                    # try to fit curve
                    try: 
                        p0 = [0.1, 0.001]  # initialise curve parameters
                        opt, pcov = curve_fit(model_func, x, y, p0)  # fit curve
                        good_curve_fit = 1
                    except: 
                        pass
                    
                    if(good_curve_fit == 1):
                        a, k = opt
                        y2 = model_func(x, a, k) # get y-values for fitted curve

                        # check distance between "curves" at end sample
                        n_rel_at_end_samp = np.sum(sample_rel_list)
                        y3 =  model_func(np.array(range(1,len(sample_rel_list)+1)), a, k)
                        est_by_curve_end_samp = np.sum(y3)
                        est_by_curve_end_samp = int(round(est_by_curve_end_samp))


                        if n_rel_at_end_samp >= des_recall*est_by_curve_end_samp:


                            # using inhom Poisson process with fitted curve as rate fn, predict total number rel docs in topic 
                            mu = (a/-k)*(math.exp(-k*n_docs)-1)  # integral model_func
                            pred_n_rel = predict_n_rel(des_prob, n_docs, mu) # predict max number rel docs (using poisson cdf)
                            des_n_rel = des_recall*pred_n_rel
                            if des_n_rel <= n_rel_at_end_samp:
                                pred_stop_n = n_rel_at_end_samp             


                    i += 1  # increase sample proportion size


                # score result 
                inhom_recall = calc_recall(rel_list, n_samp_docs)
                inhom_effort = n_samp_docs
                inhom_accept = calc_accept(inhom_recall, des_recall)
                score_dic[query_id].append((inhom_recall, inhom_effort, inhom_accept))


            else: # if not enough rel docs in min sample, stopping point is n_docs
                inhom_recall = calc_recall(rel_list, n_docs)
                inhom_effort = n_docs
                inhom_accept = calc_accept(inhom_recall, des_recall)
                score_dic[query_id].append((inhom_recall, inhom_effort, inhom_accept))
                
                
            # INHOMOGENEOUS POISSON PROCESS POWER (IP-P)
            # check topic meets initial relevance requirement
            n_samp_docs = int(round(n_docs*sample_props[0]))
            sample_rel_list = rel_list[0:n_samp_docs]  # chunck of rel list examined in sample

            # if meet size requirement run algorithm; else return n_docs as stopping point
            if (np.sum(sample_rel_list) >= min_rel_in_sample):

                windows_end_point = 0
                pred_stop_n = n_docs
                i = 0

                while (i < len(sample_props)) and (pred_stop_n > n_samp_docs):
                    sample_prop = sample_props[i]

                    n_samp_docs = int(round(n_docs*sample_props[i]))
                    sample_rel_list = rel_list[0:n_samp_docs]  # chunck of rel list examined in sample

                    # get points
                    windows = make_windows(n_windows, sample_prop, n_docs)
                    window_size = windows[0][1]

                    x,y = get_points_power(windows, window_size, sample_rel_list)  # calculate points that will be used to fit curve

                    good_curve_fit = 0
                    # try to fit curve
                    try: 
                        p0 = [0.1, 0.001]  # initialise curve parameters
                        opt, pcov = curve_fit(model_func_power, x, y, p0)  # fit curve
                        good_curve_fit = 1
                    except: 
                        pass
                    
                    if(good_curve_fit == 1):
                        a, k = opt
                        y2 = model_func_power(x, a, k) # get y-values for fitted curve

                        # check distance between "curves" at end sample
                        n_rel_at_end_samp = np.sum(sample_rel_list)
                        y3 =  model_func_power(np.array(range(1,len(sample_rel_list)+1)), a, k)
                        est_by_curve_end_samp = np.sum(y3)
                        est_by_curve_end_samp = int(round(est_by_curve_end_samp))


                        if n_rel_at_end_samp >= des_recall*est_by_curve_end_samp:


                            # using inhom Poisson process with fitted curve as rate fn, predict total number rel docs in topic 
                            #mu = (a/-k)*(math.exp(-k*n_docs)-1)  # integral model_func
                            mu = (a/(k+1))*(n_docs**(k+1)-1)  # update
                            pred_n_rel = predict_n_rel(des_prob, n_docs, mu) # predict max number rel docs (using poisson cdf)
                            des_n_rel = des_recall*pred_n_rel
                            if des_n_rel <= n_rel_at_end_samp:
                                pred_stop_n = n_rel_at_end_samp             


                    i += 1  # increase sample proportion size


                # score result 
                inhom_recall = calc_recall(rel_list, n_samp_docs)
                inhom_effort = n_samp_docs
                inhom_accept = calc_accept(inhom_recall, des_recall)
                score_dic[query_id].append((inhom_recall, inhom_effort, inhom_accept))


            else: # if not enough rel docs in min sample, stopping point is n_docs
                inhom_recall = calc_recall(rel_list, n_docs)
                inhom_effort = n_docs
                inhom_accept = calc_accept(inhom_recall, des_recall)
                score_dic[query_id].append((inhom_recall, inhom_effort, inhom_accept))


        
            # COX PROCESS EXP CURVE (CX-E)
            # check topic meets initial relevance requirement
            n_samp_docs = int(round(n_docs*sample_props[0]))
            sample_rel_list = rel_list[0:n_samp_docs]  # chuck of rel list examined in sample
            
            # if meet size requirement run algorithm; else return n_docs as stopping point
            if (np.sum(sample_rel_list) >= min_rel_in_sample):

                windows_end_point = 0
                pred_stop_n = n_docs
                i = 0

                while (i < len(sample_props)) and (pred_stop_n > n_samp_docs):
                    sample_prop = sample_props[i]

                    n_samp_docs = int(round(n_docs*sample_props[i]))
                    sample_rel_list = rel_list[0:n_samp_docs]  # chuck of rel list examined in sample

                    # get points
                    windows = make_windows(n_windows, sample_prop, n_docs)
                    window_size = windows[0][1]

                    x,y = get_points(windows, window_size, sample_rel_list)  # calculate points that will be used to fit curve

                    good_curve_fit = 0
                    # try to fit curve
                    try: 
                        p0 = [0.1, 0.001]  # initialise curve parameters
                        opt, pcov = curve_fit(model_func, x, y, p0)  # fit curve
                        good_curve_fit = 1
                    except: 
                        pass
                    
                    if(good_curve_fit == 1):
                        # marks@18/12/2020
                        # Standard deviation errors on curve fit parameters
                        perr = np.sqrt(np.diag(pcov))
                        #print("perr:", perr)
                        
                        a, k = opt 
                        y2 = model_func(x, a, k) # get y-values for fitted curve 
                        
                        # check distance between "curves" at end sample
                        n_rel_at_end_samp = np.sum(sample_rel_list)
                        y3 =  model_func(np.array(range(1,len(sample_rel_list)+1)), a, k) 
                        est_by_curve_end_samp = np.sum(y3)
                        est_by_curve_end_samp = int(round(est_by_curve_end_samp))


                        if n_rel_at_end_samp >= des_recall*est_by_curve_end_samp:

                            # marks@16/12/2020
                            # Cox process with fitted curve.
                            # Sample points from normal distribution; generate probability + predicted
                            # value from Poisson Process
                            norm_samples = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 100)
                            #print("norm_samples:",norm_samples)
                            vals = []
                            for sample in norm_samples:
                                a_val = a + sample*perr[0]
                                k_val = k - sample*perr[1]   # Note need to *subtract* from one
                                mu = (a_val/-k_val)*(math.exp(-k_val*n_docs)-1)  # integral of model_func
                                vals.append(norm.pdf(sample) * predict_n_rel(des_prob, n_docs, mu))   # predict max number rel docs (using poisson cdf)
                            # Integrate over samples to produce final prediction
                            cox_pred_n_rel = simps(vals, norm_samples)

                            des_n_rel = des_recall*cox_pred_n_rel  # alison
                            if des_n_rel <= n_rel_at_end_samp:
                                pred_stop_n = n_rel_at_end_samp             


                    i += 1  # increase sample proportion size


                # score result 
                cox_recall = calc_recall(rel_list, n_samp_docs)
                cox_effort = n_samp_docs
                cox_accept = calc_accept(cox_recall, des_recall) # alison
                score_dic[query_id].append((cox_recall, cox_effort, cox_accept))

            
            else: # if not enough rel docs in min sample, stopping point is n_docs
                cox_recall = calc_recall(rel_list, n_docs)
                cox_effort = n_docs
                cox_accept = calc_accept(cox_recall, des_recall) # alison
                score_dic[query_id].append((cox_recall, cox_effort, cox_accept))
                
        

            # COX PROCESS POWER CURVE (CX-P)
            # check topic meets initial relevance requirement
            n_samp_docs = int(round(n_docs*sample_props[0]))
            sample_rel_list = rel_list[0:n_samp_docs]  # chuck of rel list examined in sample
            
            # if meet size requirement run algorithm; else return n_docs as stopping point
            if (np.sum(sample_rel_list) >= min_rel_in_sample):

                windows_end_point = 0
                pred_stop_n = n_docs
                i = 0

                while (i < len(sample_props)) and (pred_stop_n > n_samp_docs):
                    sample_prop = sample_props[i]

                    n_samp_docs = int(round(n_docs*sample_props[i]))
                    sample_rel_list = rel_list[0:n_samp_docs]  # chuck of rel list examined in sample

                    # get points
                    windows = make_windows(n_windows, sample_prop, n_docs)
                    window_size = windows[0][1]

                    x,y = get_points_power(windows, window_size, sample_rel_list)  # update

                    good_curve_fit = 0
                    # try to fit curve
                    try: 
                        p0 = [0.1, 0.001]  # initialise curve parameters
                        opt, pcov = curve_fit(model_func_power, x, y, p0)  # update
                        good_curve_fit = 1
                    except: 
                        pass
                    
                    if(good_curve_fit == 1):
                        # marks@18/12/2020
                        # Standard deviation errors on curve fit parameters
                        perr = np.sqrt(np.diag(pcov))
                        #print("perr:", perr)
                        
                        a, k = opt 
                        y2 = model_func_power(x, a, k) # updated
                        
                        # check distance between "curves" at end sample
                        n_rel_at_end_samp = np.sum(sample_rel_list)
                        y3 =  model_func_power(np.array(range(1,len(sample_rel_list)+1)), a, k) 
                        est_by_curve_end_samp = np.sum(y3)
                        est_by_curve_end_samp = int(round(est_by_curve_end_samp))


                        if n_rel_at_end_samp >= des_recall*est_by_curve_end_samp:

                            # marks@16/12/2020
                            # Cox process with fitted curve.
                            # Sample points from normal distribution; generate probability + predicted
                            # value from Poisson Process
                            norm_samples = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 100)
                            vals = []
                            for sample in norm_samples:
                                a_val = a + sample*perr[0]
                                k_val = k - sample*perr[1]   # Note need to *subtract* from one
                                mu = (a_val/(k_val+1))*(n_docs**(k_val+1)-1)  # update
                                vals.append(norm.pdf(sample) * predict_n_rel(des_prob, n_docs, mu))   # predict max number rel docs (using poisson cdf)
                            # Integrate over samples to produce final prediction
                            cox_pred_n_rel = simps(vals, norm_samples)
              

                           
                            des_n_rel = des_recall*cox_pred_n_rel  # alison
                            if des_n_rel <= n_rel_at_end_samp:
                                pred_stop_n = n_rel_at_end_samp             


                    i += 1  # increase sample proportion size


                # score result 
                cox_power_recall = calc_recall(rel_list, n_samp_docs)
                cox_power_effort = n_samp_docs
                cox_power_accept = calc_accept(cox_power_recall, des_recall) # alison
                score_dic[query_id].append((cox_power_recall, cox_power_effort, cox_power_accept))

            
            else: # if not enough rel docs in min sample, stopping point is n_docs
                cox_power_recall = calc_recall(rel_list, n_docs)
                cox_power_effort = n_docs
                cox_power_accept = calc_accept(cox_power_recall, des_recall) # alison
                score_dic[query_id].append((cox_power_recall, cox_power_effort, cox_power_accept))
        
        
           
        
        # SCORE RESULTS
        tar_recall_vec =        [val[0][0] for val in score_dic.values()]
        km150_recall_vec =      [val[1][0] for val in score_dic.values()]
        inhom_recall_vec =      [val[2][0] for val in score_dic.values()]
        inhom_power_recall_vec= [val[3][0] for val in score_dic.values()]
        cox_recall_vec =        [val[4][0] for val in score_dic.values()]
        cox_power_recall_vec =  [val[5][0] for val in score_dic.values()]
        
        tar_eff_vec =        [val[0][1] for val in score_dic.values()]
        km150_eff_vec =      [val[1][1] for val in score_dic.values()]
        inhom_eff_vec =      [val[2][1] for val in score_dic.values()]
        inhom_power_eff_vec= [val[3][1] for val in score_dic.values()]
        cox_eff_vec =        [val[4][1] for val in score_dic.values()]
        cox_power_eff_vec =  [val[5][1] for val in score_dic.values()]
    
        topic_size_vec = [len(doc_rank_dic[query_id]) for query_id in topics_list]

        
        # ADD SCORES TO DICTIONARIES
        run_score_dic[run_name] = {}
    
        run_score_dic[run_name]['TM Rec'] = np.mean(tar_recall_vec)
        run_score_dic[run_name]['KM Rec'] = np.mean(km150_recall_vec)
        run_score_dic[run_name]['IP-E Rec'] = np.mean(inhom_recall_vec)
        run_score_dic[run_name]['CX-E Rec'] = np.mean(cox_recall_vec)
        run_score_dic[run_name]['IP-P Rec'] = np.mean(inhom_power_recall_vec)
        run_score_dic[run_name]['CX-P Rec'] = np.mean(cox_power_recall_vec)
     
        run_score_dic[run_name]['TM Eff'] =  np.sum(tar_eff_vec)
        run_score_dic[run_name]['KM Eff'] = np.sum(km150_eff_vec)
        run_score_dic[run_name]['IP-E Eff'] = np.sum(inhom_eff_vec)
        run_score_dic[run_name]['CX-E Eff'] = np.sum(cox_eff_vec)
        run_score_dic[run_name]['IP-P Eff'] = np.sum(inhom_power_eff_vec)
        run_score_dic[run_name]['CX-P Eff'] = np.sum(cox_power_eff_vec)
        run_score_dic[run_name]['OR Eff'] = np.sum(oracle_dic[run_name])
 

    # MAKE DATAFRAME OF ALL RESULTS
    df = pd.DataFrame.from_dict(run_score_dic, orient='index')
    
    return  (df, df.mean().round(2).to_dict())

In [4]:
# RUN EXPERIMENTS

def pes(eff):  # fn to calculate % of effort saved
    saving = 117562-eff
    return round(100*saving/117562,1)



def run_experiments(des_prob):  http://localhost:8888/notebooks/Documents/work_home_backups/cox_processes/SIGIR21_code/run_stopping_point_experiments.ipynb#
    print(des_prob)
    
    # get results
    results_dict_verbose = {}
    results_dict = {}
    for r in des_recalls:
        print("recall", r)
        results_dict_verbose[r], results_dict[r] = run_sp_algorithms(r, des_prob)
        
    # make results into dataframe
    df = pd.DataFrame.from_dict(results_dict, orient='index')    
    
    # add percentage effort saved cols
    df['TM PES'] = [pes(eff) for eff in df['TM Eff'].tolist()]
    df['KM PES'] = [pes(eff) for eff in df['KM Eff'].tolist()]
    df['IP-E PES'] = [pes(eff) for eff in df['IP-E Eff'].tolist()]
    df['CX-E PES'] = [pes(eff) for eff in df['CX-E Eff'].tolist()]
    df['IP-P PES'] = [pes(eff) for eff in df['IP-P Eff'].tolist()]
    df['CX-P PES'] = [pes(eff) for eff in df['CX-P Eff'].tolist()]
    df['OR PES'] = [pes(eff) for eff in df['OR Eff'].tolist()]

    # make effort cols integers
    df['KM Eff'] = df['KM Eff'].round(0).astype(int)
    df['TM Eff'] = df['TM Eff'].round(0).astype(int)
    df['IP-E Eff'] = df['IP-E Eff'].round(0).astype(int)
    df['CX-E Eff'] = df['CX-E Eff'].round(0).astype(int)
    df['IP-P Eff'] = df['IP-P Eff'].round(0).astype(int)
    df['CX-P Eff'] = df['CX-P Eff'].round(0).astype(int)
    df['OR Eff'] = df['OR Eff'].round(0).astype(int)

    df = df.sort_index()
    
    
    return df

In [5]:
df_95 = run_experiments(0.95)
df_95

0.95
recall 0.95
recall 0.9
recall 0.8
recall 0.7


,TM Rec,KM Rec,IP-E Rec,CX-E Rec,IP-P Rec,CX-P Rec,TM Eff,KM Eff,IP-E Eff,CX-E Eff,IP-P Eff,CX-P Eff,OR Eff,TM PES,KM PES,IP-E PES,CX-E PES,IP-P PES,CX-P PES,OR PES
0.70,0.92,1.0,0.97,0.98,0.99,0.99,55005,86243,54754,48290,53585,53263,7419,53.2,26.6,53.4,58.9,54.4,54.7,93.7
0.80,0.95,1.0,0.99,0.99,1.00,1.00,62867,86243,73728,80040,56840,57455,9675,46.5,26.6,37.3,31.9,51.7,51.1,91.8
0.90,0.98,1.0,1.00,1.00,1.00,1.00,83954,86243,103509,99314,61361,61711,16706,28.6,26.6,12.0,15.5,47.8,47.5,85.8
0.95,1.00,1.0,1.00,1.00,1.00,1.00,102105,86243,116521,117558,63605,63314,22637,13.1,26.6,0.9,0.0,45.9,46.1,80.7


In [6]:
print(df_95.to_latex())

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrr}
\toprule
{} &  TM Rec &  KM Rec &  IP-E Rec &  CX-E Rec &  IP-P Rec &  CX-P Rec &  TM Eff &  KM Eff &  IP-E Eff &  CX-E Eff &  IP-P Eff &  CX-P Eff &  OR Eff &  TM PES &  KM PES &  IP-E PES &  CX-E PES &  IP-P PES &  CX-P PES &  OR PES \\
\midrule
0.70 &    0.92 &     1.0 &      0.97 &      0.98 &      0.99 &      0.99 &   55005 &   86243 &     54754 &     48290 &     53585 &     53263 &    7419 &    53.2 &    26.6 &      53.4 &      58.9 &      54.4 &      54.7 &    93.7 \\
0.80 &    0.95 &     1.0 &      0.99 &      0.99 &      1.00 &      1.00 &   62867 &   86243 &     73728 &     80040 &     56840 &     57455 &    9675 &    46.5 &    26.6 &      37.3 &      31.9 &      51.7 &      51.1 &    91.8 \\
0.90 &    0.98 &     1.0 &      1.00 &      1.00 &      1.00 &      1.00 &   83954 &   86243 &    103509 &     99314 &     61361 &     61711 &   16706 &    28.6 &    26.6 &      12.0 &      15.5 &      47.8 &      47.5 &    85.8 \\
0.95 &    1.00 & 

In [7]:
df_80 = run_experiments(0.80)
df_80

0.8
recall 0.95
recall 0.9
recall 0.8
recall 0.7


,TM Rec,KM Rec,IP-E Rec,CX-E Rec,IP-P Rec,CX-P Rec,TM Eff,KM Eff,IP-E Eff,CX-E Eff,IP-P Eff,CX-P Eff,OR Eff,TM PES,KM PES,IP-E PES,CX-E PES,IP-P PES,CX-P PES,OR PES
0.70,0.85,1.0,0.97,0.97,0.99,0.99,43396,86243,46783,47622,51479,51236,7419,63.1,26.6,60.2,59.5,56.2,56.4,93.7
0.80,0.90,1.0,0.98,0.98,0.99,0.99,52417,86243,56051,56050,54948,54758,9675,55.4,26.6,52.3,52.3,53.3,53.4,91.8
0.90,0.96,1.0,0.99,0.99,1.00,1.00,66476,86243,79563,83510,58619,59113,16706,43.5,26.6,32.3,29.0,50.1,49.7,85.8
0.95,0.99,1.0,1.00,0.99,1.00,1.00,86381,86243,95235,92819,59703,60043,22637,26.5,26.6,19.0,21.0,49.2,48.9,80.7


In [8]:
print(df_80.to_latex())

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrr}
\toprule
{} &  TM Rec &  KM Rec &  IP-E Rec &  CX-E Rec &  IP-P Rec &  CX-P Rec &  TM Eff &  KM Eff &  IP-E Eff &  CX-E Eff &  IP-P Eff &  CX-P Eff &  OR Eff &  TM PES &  KM PES &  IP-E PES &  CX-E PES &  IP-P PES &  CX-P PES &  OR PES \\
\midrule
0.70 &    0.85 &     1.0 &      0.97 &      0.97 &      0.99 &      0.99 &   43396 &   86243 &     46783 &     47622 &     51479 &     51236 &    7419 &    63.1 &    26.6 &      60.2 &      59.5 &      56.2 &      56.4 &    93.7 \\
0.80 &    0.90 &     1.0 &      0.98 &      0.98 &      0.99 &      0.99 &   52417 &   86243 &     56051 &     56050 &     54948 &     54758 &    9675 &    55.4 &    26.6 &      52.3 &      52.3 &      53.3 &      53.4 &    91.8 \\
0.90 &    0.96 &     1.0 &      0.99 &      0.99 &      1.00 &      1.00 &   66476 &   86243 &     79563 &     83510 &     58619 &     59113 &   16706 &    43.5 &    26.6 &      32.3 &      29.0 &      50.1 &      49.7 &    85.8 \\
0.95 &    0.99 & 